# Setup

In [1]:
from setup import *

In [2]:
import osmnx
from pqdm.processes import pqdm
from shapely.geometry import LineString, Point

In [3]:
aus = City('Austin, TX')
cam = City('Cambridge, MA')
sea = City('Seattle, WA')
tor = City('Toronto, Ontario')

---
# Boundaries

## City boundary

In [4]:
def save_osm_city_boundary(city):
    city.save('boundary', osmnx.geocode_to_gdf(city.geocode)[['geometry']].to_crs(CRS_DEG))

**AUSTIN**

[**CAMBRIDGE**](https://www.cambridgema.gov/GIS/gisdatadictionary/Boundary/BOUNDARY_CityBoundary)

[**SEATTLE**](https://geo.wa.gov/datasets/WSDOT::wsdot-city-limits/about)

[**TORONTO**](https://open.toronto.ca/dataset/regional-municipal-boundary/)

## Roads

[**AUSTIN**](https://data.austintexas.gov/Locations-and-Maps/Street-Centerline/m5w3-uea6)

[**CAMBRIDGE**](https://www.cambridgema.gov/GIS/gisdatadictionary/Trans/TRANS_Centerlines)

[**SEATTLE**](https://data-seattlecitygis.opendata.arcgis.com/datasets/SeattleCityGIS::seattle-streets/about)

[**TORONTO**](https://open.toronto.ca/dataset/toronto-centreline-tcl/)

## Sidewalks

[**AUSTIN**](https://data.austintexas.gov/Locations-and-Maps/Sidewalks/pc5y-5bpw)

In [5]:
aus.load('sidewalks').kind.value_counts().sum()

343773

[**CAMBRIDGE**](https://www.cambridgema.gov/GIS/gisdatadictionary/Trans/TRANS_SidewalkCenterlines)

[**SEATTLE**](https://data-seattlecitygis.opendata.arcgis.com/datasets/SeattleCityGIS::sidewalks/about)

[**TORONTO**](https://open.toronto.ca/dataset/pedestrian-network/)

## Crosswalks

[**AUSTIN**](https://data.austintexas.gov/Transportation-and-Mobility/TRANSPORTATION-markings_short_line/3p2i-pqdc)

[**CAMBRIDGE**](https://www.cambridgema.gov/GIS/gisdatadictionary/Trans/TRANS_SidewalkCenterlines)

[**SEATTLE**](https://data-seattlecitygis.opendata.arcgis.com/datasets/SeattleCityGIS::marked-crosswalks/about)

[**TORONTO**](https://open.toronto.ca/dataset/pedestrian-network/)

## Snap crosswalks to sidewalks

**AUSTIN**

**SEATTLE** (**Not complete**)

---
# Candidate crosswalks

## Generate sidewalk endpoints

In [6]:
def make_candidate_xwalk_endpoints(city, buffer=20):
    """
    Generate a list of sidewalk endpoints within road intersections that will 
    then be used to generate pair-wise combinations.
    """
    # load base layers
    roads = (city.load('roads').dropna(subset='geometry')
             .explode(index_parts=True).reset_index(drop=True).rename_axis('road_id'))
    sw = city.load('sidewalks')
    xw = city.load('crosswalks')
    # find intersections (as road endpoints)
    ixns = Pdf([dict(zip(['src','trg'], [x.coords[0], x.coords[-1]])) for x 
                in roads.geometry]).assign(road_id=roads.index)
    ixns = (ixns.melt('road_id').groupby('value')['road_id'].agg([list, 'count']).query('count > 1')
            .rename_axis('xy')['list'].rename('road_id').reset_index().rename_axis('ixn_id'))
    ixns = Gdf(ixns.assign(geometry=gpd.GeoSeries([Point(x) for x in ixns.pop('xy')], crs=CRS_DEG)))
    # create intersection buffers
    ixn_buf = ixns.assign(geometry=ixns.to_crs(CRS_M).buffer(buffer).to_crs(CRS_DEG))
    # sidewalk segments within intersection buffers
    sw_in = (gpd.sjoin(sw.rename_axis('sw_id'), ixn_buf.reset_index()).drop(
        columns='index_right').groupby('sw_id')['ixn_id'].agg(list).reset_index())
    # both endpoints of sidewalks in buffer
    df = sw.loc[sw_in['sw_id']].rename_axis('sw_id')
    src, trg = zip(*[(x.coords[0], x.coords[-1]) for x in df.geometry])
    df = pd.concat([Seq(src, index=df.index), Seq(trg, index=df.index)]).reset_index()
    df = df.groupby(0)['sw_id'].agg(list).rename_axis('xy').reset_index().rename_axis('sw_pt_id')
    sw_in_pts = Gdf(df.assign(geometry=gpd.GeoSeries([Point(x) for x in df.pop('xy')], crs=CRS_DEG)))
    # map sidewalk endpoint to intersection
    sw_pt2ixn = gpd.sjoin(sw_in_pts, ixn_buf.reset_index())['ixn_id'].reset_index()
    # sidewalk endpoints within intersections
    sw_pts_in = gpd.sjoin(sw_in_pts, ixn_buf).drop(columns='index_right')
    # assign sidewalks to nearest roads
    sw2road = (sw_in.explode('ixn_id').merge(ixns[['road_id']], on='ixn_id').explode('road_id')
               .merge(sw_in_pts.explode('sw_id')['sw_id'].reset_index(), on='sw_id'))
    sw2road['dist'] = (sw_in_pts.to_crs(CRS_M).loc[sw2road['sw_pt_id'], 'geometry'].distance(
        roads.to_crs(CRS_M).loc[sw2road['road_id'], 'geometry'], align=False).values)
    sw2road = (sw2road.groupby(['sw_id','road_id'])['dist'].mean().reset_index().sort_values('dist')
           .groupby('sw_id').head(1).reset_index()[['sw_id','road_id']])
    # get attributes of each candidate crosswalk endpoint (nearest road, swalk & intersection)
    xw_pts = (sw_in_pts.explode('sw_id')['sw_id'].reset_index()
              .merge(sw_in, on='sw_id')
              .merge(sw2road, on='sw_id').explode('ixn_id')
              .merge(ixns['road_id'].rename('rid2'), on='ixn_id').explode('rid2')
              .merge(sw_pts_in.reset_index()[['sw_pt_id']], on='sw_pt_id')
              .query('road_id == rid2').reset_index(drop=True)
              .merge(sw_pt2ixn.rename(columns={'ixn_id':'ixn_uid'}), on='sw_pt_id')
              .drop(columns='rid2').astype(np.int32))
    return sw_in_pts, xw_pts

sw_pts, candi_pts = make_candidate_xwalk_endpoints(cam)
candi_pts.disp(3); pass

9,692 rows x 5 cols; Memory: 0.3 MiB


,sw_pt_id,sw_id,ixn_id,road_id,ixn_uid
,<int32>,<int32>,<int32>,<int32>,<int32>
0,2,283,0,1302,0
1,3,1031,0,2575,0
2,4,1031,0,2575,0


## Create eligible endpoint combinations

In [7]:
def make_xwalk_combinations(grp_pair):
    ixn_id, df = grp_pair
    return (df.merge(df, how='cross').query(' and '.join([
        'sw_pt_id_x != sw_pt_id_y',
        'sw_id_x != sw_id_y',
        'road_id_x == road_id_y',
        'ixn_uid_x == ixn_uid_y'
    ])).rename(columns={'ixn_id_x': 'ixn_id', 'road_id_x': 'road_id'})
            .drop(columns=['ixn_id_y', 'ixn_uid_x', 'ixn_uid_y', 'road_id_y']))

In [8]:
def make_candidate_xwalk_segments(sw_pts, candi_pts, roads, xwalks):
    # generate crosswalk combinations from eligible sidewalk endpoints
    combs = (pd.concat(pqdm(candi_pts.groupby('ixn_id'), make_xwalk_combinations,
                            n_jobs=40, total=candi_pts['ixn_id'].nunique()))
             .reset_index(drop=True))
    # create candidate crosswalk linestrings from sidewalk endpoint pairs
    xw = (combs.merge(sw_pts['geometry'].rename('src').rename_axis('sw_pt_id_x').reset_index())
          .merge(sw_pts['geometry'].rename('trg').rename_axis('sw_pt_id_y').reset_index()))
    xw['geometry'] = gpd.GeoSeries([LineString(x) for x in zip(xw['src'], xw['trg'])])
    xw = Gdf(xw, crs=CRS_DEG).assign(**{c: xw[c].apply(lambda x: x.coords[0]) for c in ['src','trg']})
    # filter candidates that intersect their road and get their endpoints
    xw = xw[xw.geometry.intersects(roads.loc[xw['road_id'], 'geometry'], align=False)]
    # identify and remove existing crosswalks from the candidates
    xwalks = Pdf([dict(zip(['src','trg'], [x.coords[0], x.coords[-1]])) for x in xwalks.geometry])
    common_xw_ids = pd.concat([
        xw.reset_index().merge(xwalks, on=('src','trg'))['index'],
        xw.reset_index().merge(xwalks, left_on=('src','trg'), right_on=('trg','src'))['index']
    ]).drop_duplicates().values
    xw = xw.loc[list(set(xw.index) - set(common_xw_ids))].drop(columns=['src','trg']).rename_axis('id')
    # find the shortest crosswalk segment at each interesection leg
    xw['len'] = xw.to_crs(CRS_M).geometry.length
    xw = xw.loc[xw.sort_values('len').groupby(['ixn_id','road_id']).head(1).reset_index()['id']]
    return xw.reset_index(drop=True).rename_axis('id')
    
%time make_candidate_xwalk_segments(sw_pts, candi_pts, cam.roads, cam.crosswalks).disp(); pass

QUEUEING TASKS | :   0%|          | 0/1205 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1205 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1205 [00:00<?, ?it/s]

733 rows x 8 cols; Memory: 0.0 MiB; CRS: EPSG:4326


,sw_pt_id_x,sw_id_x,ixn_id,road_id,sw_pt_id_y,sw_id_y,geometry,len
id,<int32>,<int32>,<int32>,<int32>,<int32>,<int32>,<geometry>,<float64>
0,3009,10789,1035,1329,3002,10740,"LINESTRING (-71.105623 42.359573, -71.105642 4...",3.459243


CPU times: user 1.75 s, sys: 711 ms, total: 2.46 s
Wall time: 2.4 s


## Overall

In [9]:
def make_candidate_xwalks(city, save=True):
    sw_pts, candi_pts = make_candidate_xwalk_endpoints(city)
    roads, cur_xwalks = city.load('roads'), city.load('crosswalks')
    candi = make_candidate_xwalk_segments(sw_pts, candi_pts, roads, cur_xwalks)
    if save:
        city.save('candidate_crosswalks', candi)
    return candi

**AUSTIN**

**CAMBRIDGE**

**TORONTO**

---
# Base pednet

In [10]:
def create_base_pednet(city, save=True):
    sw = city.load('sidewalks').rename_axis('orig_id').assign(is_xwalk=False)
    if 'exists' not in sw.columns: sw['exists'] = True
    xw = city.load('crosswalks').rename_axis('orig_id').assign(is_xwalk=True, exists=True)
    candi = (city.load('candidate_crosswalks')[['geometry']].rename_axis('orig_id')
             .assign(is_xwalk=True, exists=False))
    E = pd.concat([sw.reset_index(), xw.reset_index(), candi.reset_index()])
    E['len'] = E.to_crs(CRS_M).geometry.length
    E = E[E['len'] > 0].reset_index(drop=True).rename_axis('eid').reset_index()
    E['src'], E['trg'] = zip(*[(x.coords[0], x.coords[-1]) for x in tqdm(E.geometry)])
    V = Seq(list(set(E['src']) | set(E['trg'])), name='xy').rename_axis('vid').reset_index()
    E = E.merge(V.rename(columns={'vid':'src_vid', 'xy':'src'}))
    E = E.merge(V.rename(columns={'vid':'trg_vid', 'xy':'trg'}))
    E = E.drop(columns=['src','trg']).set_index('eid').sort_index()
    E = E.astype({'src_vid': np.int32, 'trg_vid': np.int32, 'len': np.float32})
    V['geometry'] = gpd.GeoSeries([Point(x) for x in V['xy']], crs=CRS_DEG)
    V['x'], V['y'] = zip(*V['xy'])
    V = Gdf(V.set_index('vid'))[['x','y','geometry']]
    if save:
        city.save('full_pednet_nodes', V)
        city.save('full_pednet_edges', E)
    return V, E

# %time x = create_base_pednet(aus); x

**AUSTIN**

**CAMBRIDGE**

**TORONTO**